# L-Layer, Single Neuron Testing
## Architecture

```json
{
    "epochs": 3,
    "layers": 1,
    "activations": {
        "layer1": "sigmoid"
    },
    "neurons": {
        "layer1": 1
    },
    "weight": 0,
    "bias": 0,
    "learning_rate": 0.005
}
```

---

## LaunchLambda

In [1]:
# Import Libraries needed by the Lambda Function
import numpy as np
import h5py
import scipy
import os
from os import environ
import json
from json import dumps, loads
from boto3 import client, resource, Session
import botocore
import uuid
import io
import redis
from redis import StrictRedis as redis

# Global Variables
#rgn = environ['Region']
rgn = 'us-west-2'
s3_client = client('s3', region_name=rgn) # S3 access
s3_resource = resource('s3')
sns_client = client('sns', region_name=rgn) # SNS
redis_client = client('elasticache', region_name=rgn)
#Retrieve the Elasticache Cluster endpoint
cc = redis_client.describe_cache_clusters(ShowCacheNodeInfo=True)
endpoint = cc['CacheClusters'][0]['CacheNodes'][0]['Endpoint']['Address']
lambda_client = client('lambda', region_name=rgn) # Lambda invocations

# Helper Functions
def get_arns(function_name):
    """
    Return the ARN for the LNN Functions.
    Note: This addresses circular dependency issues in CloudFormation
    """
    function_list = lambda_client.list_functions()
    function_arn = None
    for function in function_list['Functions']:
        if function['FunctionName'] == function_name:
            function_arn = function['FunctionArn']
            break
    return function_arn

def publish_sns(sns_message):
    """
    Publish message to the master SNS topic.

    Arguments:
    sns_message -- the Body of the SNS Message
    """

    print("Publishing message to SNS topic...")
    sns_client.publish(TargetArn=environ['SNSArn'], Message=sns_message)
    return

def to_cache(endpoint, obj, name):
    """
    Serializes multiple data type to ElastiCache and returns
    the Key.
    
    Arguments:
    endpoint -- The ElastiCache endpoint
    obj -- the object to srialize. Can be of type:
            - Numpy Array
            - Python Dictionary
            - String
            - Integer
    name -- Name of the Key
    
    Returns:
    key -- For each type the key is made up of {name}|{type} and for
           the case of Numpy Arrays, the Length and Widtch of the 
           array are added to the Key.
    """
    
    # Test if the object to Serialize is a Numpy Array
    if 'numpy' in str(type(obj)):
        array_dtype = str(obj.dtype)
        if len(obj.shape) == 0:
            length = 0
            width = 0
        else:
            length, width = obj.shape
        # Convert the array to string
        val = obj.ravel().tostring()
        # Create a key from the name and necessary parameters from the array
        # i.e. {name}|{type}#{length}#{width}
        key = '{0}|{1}#{2}#{3}'.format(name, array_dtype, length, width)
        # Store the binary string to Redis
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a string
    elif type(obj) is str:
        key = '{0}|{1}'.format(name, 'string')
        val = obj
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is an integer
    elif type(obj) is int:
        key = '{0}|{1}'.format(name, 'int')
        # Convert to a string
        val = str(obj)
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a dictionary
    elif type(obj) is dict:
        # Convert the dictionary to a String
        val = json.dumps(obj)
        key = '{0}|{1}'.format(name, 'json')
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    else:
        sns_message = "`to_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported serialization type")
        raise

def from_cache(endpoint, key):
    """
    De-serializes binary object from ElastiCache by reading
    the type of object from the name and converting it to
    the appropriate data type
    
    Arguments:
    endpoint -- ElastiCacheendpoint
    key -- Name of the Key to retrieve the object
    
    Returns:
    obj -- The object converted to specifed data type
    """
    
    # Check if the Key is for a Numpy array containing
    # `float64` data types
    if 'float64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        if int(length) == 0:
            obj = np.float64(np.fromstring(val))
        else:
            obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a Numpy array containing
    # `int64` data types
    elif 'int64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a json type
    elif 'json' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return json.loads(obj)
    # Chec if the Key is an integer
    elif 'int' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return int(obj)
    # Check if the Key is a string
    elif 'string' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return obj
    else:
        sns_message = "`from_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported de-serialization type")
        raise

def name2str(obj, namespace):
    """
    Converts the name of the numpy array to string
    
    Arguments:
    obj -- Numpy array object
    namespace -- dictionary of the current global symbol table
    
    Return:
    List of the names of the Numpy arrays
    """
    return [name for name in namespace if namespace[name] is obj]

def vectorize(x_orig):
    """
    Vectorize the image data into a matrix of column vectors
    
    Argument:
    x_orig -- Numpy array of image data
    
    Return:
    Reshaped/Transposed Numpy array
    """
    return x_orig.reshape(x_orig.shape[0], -1).T

def standardize(x_orig):
    """
    Standardize the input data
    
    Argument:
    x_orig -- Numpy array of image data
    
    Return:
    Call to `vectorize()`, stndrdized Numpy array of image data
    """
    return vectorize(x_orig) / 255

def initialize_data(endpoint, parameters):
    """
    Extracts the training and testing data from S3, flattens, 
    standardizes and then dumps the data to ElastiCache 
    for neurons to process as layer a^0
    """
    
    # Load main dataset
    dataset = h5py.File('/tmp/datasets.h5', "r")
    
    # Create numpy arrays from the various h5 datasets
    train_set_x_orig = np.array(dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(dataset["train_set_y"][:]) # train set labels
    test_set_x_orig = np.array(dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(dataset["test_set_y"][:]) # test set labels
    
    # Reshape labels
    train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    # Preprocess inputs
    train_set_x = standardize(train_set_x_orig)
    test_set_x = standardize(test_set_x_orig)

    # Create necessary keys for the data in ElastiCache
    data_keys = {} # Dictionary for the hask keys of the data set
    dims = {} # Dictionary of data set dimensions
    a_list = [train_set_x, train_set_y, test_set_x, test_set_y]
    a_names = [] # Placeholder for array names
    for i in range(len(a_list)):
        # Create a lis of the names of the numpy arrays
        a_names.append(name2str(a_list[i], locals()))
    for j in range(len(a_list)):
        # Dump the numpy arrays to ElastiCache
        data_keys[str(a_names[j][0])] = to_cache(endpoint=endpoint, obj=a_list[j], name=a_names[j][0])
        # Append the array dimensions to the list
        dims[str(a_names[j][0])] = a_list[j].shape
    
    # Initialize A0 and Y names from `train_setx`` and train_set_y`
    data_keys['A0'] = to_cache(endpoint=endpoint, obj=train_set_x, name='A0')
    data_keys['Y'] = to_cache(endpoint=endpoint, obj=train_set_y, name='Y')
    
    # Initialize weights to zero for single layer
    dim = dims.get('train_set_x')[0]
    weights = np.zeros((dim, 1))
    # Store the initial weights in ElastiCache
    data_keys['weights'] = to_cache(endpoint=endpoint, obj=weights, name='weights')
        
    # Initialize Bias to zero for single layer
    bias = 0
    # Store the bias in ElastiCache
    data_keys['bias'] = to_cache(endpoint=endpoint, obj=bias, name='bias')
   
    # Initialize training example size
    m = train_set_x.shape[1]
    data_keys['m'] = to_cache(endpoint, obj=m, name='m')
    
    # Initialize the results tracking object
    results = {}
    data_keys['results'] = to_cache(endpoint, obj=results, name='results')

    # Initialize gradient tracking object for each layer
    grads = {}
    for l in range(1, parameters['layers']+1):
        layer_name = 'layer' + str(l)
        grads[layer_name] = {}
    data_keys['grads'] = to_cache(endpoint=endpoint, obj=grads, name='grads')
        
    return data_keys, [j for i in a_names for j in i], dims

def launch_handler(event, context):
    # Retrieve datasets and setting from S3
    input_bucket = s3_resource.Bucket(str(event['Records'][0]['s3']['bucket']['name']))
    dataset_key = str(event['Records'][0]['s3']['object']['key'])
    settings_key = dataset_key.split('/')[-2] + '/parameters.json'
    try:
        input_bucket.download_file(dataset_key, '/tmp/datasets.h5')
        input_bucket.download_file(settings_key, '/tmp/parameters.json')
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            sns_message = "Error downloading input data from S3, S3 object does not exist"
            publish_sns(sns_message)
        else:
            raise
    
    # Extract the neural network parameters
    with open('/tmp/parameters.json') as parameters_file:
        parameters = json.load(parameters_file)
    
    # Get the ARNs for the TrainerLambda and NeuronLambda
    parameters['ARNs'] = {
        'TrainerLambda': get_arns('TrainerLambda'),
        'NeuronLambda': get_arns('NeuronLambda')
    }

    # Build in additional neural network parameters
    # Input data sets and data set parameters
    parameters['s3_bucket'] = event['Records'][0]['s3']['bucket']['name']
    parameters['data_keys'],\
    parameters['input_data'],\
    parameters['dims'] = initialize_data(
        endpoint=endpoint,
        parameters=parameters
    )
    
    # Initialize payload to `TrainerLambda`
    payload = {}
    # Initialize the overall state
    payload['state'] = 'start'
    # Dump the parameters to ElastiCache
    payload['parameter_key'] = to_cache(endpoint, obj=parameters, name='parameters')
    #payload['endpoint'] = endpoint
    # Prepare the payload for `TrainerLambda`
    payloadbytes = dumps(payload)
    
    # Debug Statements
    print("Complete Neural Network Settings: \n")
    print(dumps(parameters, indent=4, sort_keys=True))
    print("Payload to be sent to TrainerLambda: \n" + dumps(payload))

    
    """
    # Invoke TrainerLambda for next layer
    try:
        response = lambda_client.invoke(
            FunctionName=parameters['ARNs']['TrainerLambda'],
            InvocationType='Event',
            Payload=payloadbytes
            )
    except botocore.exceptions.ClientError as e:
        sns_message = "Errors occurred invoking TrainerLambda from LaunchLambda."
        sns_message += "\nError:\n" + e
        sns_message += "\nCurrent Payload:\n" +  dumps(payload, indent=4, sort_keys=True)
        publish_sns(sns_message)
        print(e)
        raise
    print(response)
    """

    return

In [2]:
context = ''
# Simulate S3 event trigger data
event = {
    "Records": [
        {
            "eventVersion": "2.0",
            "eventTime": "1970-01-01T00:00:00.000Z",
            "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
             },
            "s3": {
                "configurationId": "testConfigRule",
                "object": {
                    "eTag": "0123456789abcdef0123456789abcdef",
                    "sequencer": "0A1B2C3D4E5F678901",
                    "key": "training_input/datasets.h5",
                    "size": 1024
                },
                "bucket": {
                    "arn": "arn:aws:s3:::lnn",
                    "name": "lnn",
                    "ownerIdentity": {
                        "principalId": "EXAMPLE"
                    }
                },
                "s3SchemaVersion": "1.0"
            },
            "responseElements": {
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH",
                "x-amz-request-id": "EXAMPLE123456789"
            },
            "awsRegion": "us-west-2",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "EXAMPLE"
            },
            "eventSource": "aws:s3"
        }
    ]
}

In [3]:
launch_handler(event, context)


Complete Neural Network Settings: 

{
    "ARNs": {
        "NeuronLambda": "arn:aws:lambda:us-west-2:722812380636:function:NeuronLambda",
        "TrainerLambda": "arn:aws:lambda:us-west-2:722812380636:function:TrainerLambda"
    },
    "activations": {
        "layer1": "sigmoid"
    },
    "bias": 0,
    "data_keys": {
        "A0": "A0|float64#12288#209",
        "Y": "Y|int64#1#209",
        "bias": "bias|int",
        "grads": "grads|json",
        "m": "m|int",
        "results": "results|json",
        "test_set_x": "test_set_x|float64#12288#50",
        "test_set_y": "test_set_y|int64#1#50",
        "train_set_x": "train_set_x|float64#12288#209",
        "train_set_y": "train_set_y|int64#1#209",
        "weights": "weights|float64#12288#1"
    },
    "dims": {
        "test_set_x": [
            12288,
            50
        ],
        "test_set_y": [
            1,
            50
        ],
        "train_set_x": [
            12288,
            209
        ],
        "train_

---
## Epoch 0

In [4]:
# Import Libraries needed by the Lambda Function
import numpy as np
import h5py
import scipy
import os
from os import environ
import json
from json import dumps, loads
from boto3 import client, resource, Session
import botocore
import uuid
import io
import redis
from redis import StrictRedis as redis

# Global Variables
#rgn = environ['Region']
s3_client = client('s3', region_name=rgn) # S3 access
s3_resource = resource('s3')
sns_client = client('sns', region_name=rgn) # SNS
lambda_client = client('lambda', region_name=rgn) # Lambda invocations
redis_client = client('elasticache', region_name=rgn) # ElastiCache
# Retrieve the Elasticache Cluster endpoint
cc = redis_client.describe_cache_clusters(ShowCacheNodeInfo=True)
endpoint = cc['CacheClusters'][0]['CacheNodes'][0]['Endpoint']['Address']
cache = redis(host=endpoint, port=6379, db=0)

# Helper Functions
def publish_sns(sns_message):
    """
    Publish message to the master SNS topic.

    Arguments:
    sns_message -- the Body of the SNS Message
    """

    print("Publishing message to SNS topic...")
    sns_client.publish(TargetArn=environ['SNSArn'], Message=sns_message)
    return

def numpy2s3(array, name, bucket):
    """
    Serialize a Numpy array to S3 without using local copy
    
    Arguments:
    array -- Numpy array of any shape
    name -- filename on S3
    """
    f_out = io.BytesIO()
    np.save(f_out, array)
    try:
        s3_client.put_object(Key=name, Bucket=bucket, Body=f_out.getvalue(), ACL='bucket-owner-full-control')
    except botocore.exceptions.ClientError as e:
        sns_message = "The following error occurred while running `numpy2s3`:\n" + e
        publish_sns(sns_message)
        raise

def to_cache(endpoint, obj, name):
    """
    Serializes multiple data type to ElastiCache and returns
    the Key.
    
    Arguments:
    endpoint -- The ElastiCache endpoint
    obj -- the object to srialize. Can be of type:
            - Numpy Array
            - Python Dictionary
            - String
            - Integer
    name -- Name of the Key
    
    Returns:
    key -- For each type the key is made up of {name}|{type} and for
           the case of Numpy Arrays, the Length and Widtch of the 
           array are added to the Key.
    """
    # Test if the object to Serialize is a Numpy Array
    if 'numpy' in str(type(obj)):
        array_dtype = str(obj.dtype)
        if len(obj.shape) == 0:
            length = 0
            width = 0
        else:
            length, width = obj.shape
        # Convert the array to string
        val = obj.ravel().tostring()
        # Create a key from the name and necessary parameters from the array
        # i.e. {name}|{type}#{length}#{width}
        key = '{0}|{1}#{2}#{3}'.format(name, array_dtype, length, width)
        # Store the binary string to Redis
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a string
    elif type(obj) is str:
        key = '{0}|{1}'.format(name, 'string')
        val = obj
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is an integer
    elif type(obj) is int:
        key = '{0}|{1}'.format(name, 'int')
        # Convert to a string
        val = str(obj)
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a dictionary
    elif type(obj) is dict:
        # Convert the dictionary to a String
        val = json.dumps(obj)
        key = '{0}|{1}'.format(name, 'json')
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    else:
        sns_message = "`to_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported serialization type")
        raise

def from_cache(endpoint, key):
    """
    De-serializes binary object from ElastiCache by reading
    the type of object from the name and converting it to
    the appropriate data type
    
    Arguments:
    endpoint -- ElastiCacheendpoint
    key -- Name of the Key to retrieve the object
    
    Returns:
    obj -- The object converted to specifed data type
    """
    # Check if the Key is for a Numpy array containing
    # `float64` data types
    if 'float64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        if int(length) == 0:
            obj = np.float64(np.fromstring(val))
        else:
            obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a Numpy array containing
    # `int64` data types
    elif 'int64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a json type
    elif 'json' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return json.loads(obj)
    # Chec if the Key is an integer
    elif 'int' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return int(obj)
    # Check if the Key is a string
    elif 'string' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return obj
    else:
        sns_message = "`from_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported de-serialization type")
        raise

def start_epoch(epoch, layer, parameter_key):
    """
    Starts a new epoch and configures the necessary state tracking objcts.
    
    Arguments:
    epoch -- Integer representing the "current" epoch.
    layer -- Integer representing the current hidden layer.
    """
    # Initialize the results object for the new epoch
    parameters = from_cache(endpoint=endpoint, key=parameter_key)
    
    # Add current epoch to results
    epoch2results = from_cache(endpoint=endpoint, key=parameters['data_keys']['results'])
    epoch2results['epoch' + str(epoch)] = {}
    parameters['data_keys']['results'] = to_cache(endpoint=endpoint, obj=epoch2results, name='results')
   
    # Update paramaters with this functions data
    parameters['epoch'] = epoch
    parameters['layer'] = layer
    parameter_key = to_cache(endpoint=endpoint, obj=parameters, name='parameters')
    
    # Start forwardprop
    propogate(direction='forward', epoch=epoch, layer=layer+1, parameter_key=parameter_key)

def end(parameter_key):
    """
    Finishes the oveall training sequence and saves the "optmized" 
    weights and bias to S3, for the prediction aplication.
    
    Arguments:
    parameter_key -- The ElastiCache key for the current set of state parameters.
    """
    # Get the latest parameters
    parameters = from_cache(
        endpoint=endpoint,
        key=parameter_key
    )

    # Get the results key
    final_results = from_cache(
        endpoint=endpoint,
        key=parameters['data_keys']['results']
    )
    # Upload the final results to S3
    bucket = parameters['s3_bucket']
    results_obj = s3_resource.Object(bucket,'training_results/results.json')
    try:
        results_obj.put(Body=json.dumps(final_results))
    except botocore.exceptions.ClientError as e:
        print(e)
        raise
    
    # Get the final Weights and Bias
    weights = from_cache(
        endpoint=endpoint,
        key=parameters['data_keys']['weights']
    )
    bias = from_cache(
        endpoint=endpoint,
        key=parameters['data_keys']['bias']
    )
    
    # Put the weights and bias onto S3 for prediction
    numpy2s3(array=weights, name='predict_input/weights', bucket=bucket)
    numpy2s3(array=bias, name='predict_input/bias', bucket=bucket)

    sns_message = "Training Completed Successfully!\n" + dumps(final_results, indent=4, sort_keys=True)

    return

def propogate(direction, epoch, layer, parameter_key):
    """
    Determines the amount of "hidden" units based on the layer and loops
    through launching the necessary `NeuronLambda` functions with the 
    appropriate state. Each `NeuronLambda` implements the cost function 
    OR the gradients depending on the direction.

    Arguments:
    direction -- The current direction of the propogation, either `forward` or `backward`.
    epoch -- Integer representing the "current" epoch to close out.
    layer -- Integer representing the current hidden layer.
    """    
    # Get the parameters for the layer
    parameters = from_cache(endpoint=endpoint, key=parameter_key)
    num_hidden_units = parameters['neurons']['layer' + str(layer)]
    
    # Build the NeuronLambda payload
    payload = {}
    # Add the parameters to the payload
    payload['state'] = direction
    payload['parameter_key'] = parameter_key
    payload['epoch'] = epoch
    payload['layer'] = layer

    # Determine process based on direction
    if direction == 'forward':
        # Launch Lambdas to propogate forward
        # Prepare the payload for `NeuronLambda`
        # Update parameters with this function's updates
        parameters['epoch'] = epoch
        parameters['layer'] = layer
        payload['parameter_key'] = to_cache(endpoint=endpoint, obj=parameters, name='parameters')

        print("Starting Forward Propogation for epoch " + str(epoch) + ", layer " + str(layer))

        for i in range(1, num_hidden_units + 1):
            # Prepare the payload for `NeuronLambda`
            payload['id'] = i
            if i == num_hidden_units:
                payload['last'] = "True"
            else:
                payload['last'] = "False"
            payload['activation'] = parameters['activations']['layer' + str(layer)]
            payloadbytes = dumps(payload)
            print("Payload to be sent NeuronLambda: \n" + dumps(payload, indent=4, sort_keys=True))

            """
            # Invoke NeuronLambdas for next layer
            try:
                response = lambda_client.invoke(
                    FunctionName=parameters['ARNs']['NeuronLambda'],
                    InvocationType='Event',
                    Payload=payloadbytes
                )
            except botocore.exceptions.ClientError as e:
                sns_message = "Errors occurred invoking Neuron Lambda from TrainerLambda."
                sns_message += "\nError:\n" + e
                sns_message += "\nCurrent Payload:\n" +  dumps(payload, indent=4, sort_keys=True)
                publish_sns(sns_message)
                print(e)
                raise
            print(response)
            """
        
        return
    
    elif direction == 'backward':
        # Launch Lambdas to propogate backward        
        # Prepare the payload for `NeuronLambda`
        # Update parameters with this functions updates
        parameters['epoch'] = epoch
        parameters['layer'] = layer
        payload['parameter_key'] = to_cache(endpoint=endpoint, obj=parameters, name='parameters')

        print("Starting Backward Propogation for epoch " + str(epoch) + ", layer " + str(layer))

        for i in range(1, num_hidden_units + 1):
            # Prepare the payload for `NeuronLambda`
            payload['id'] = i
            if i == num_hidden_units:
                payload['last'] = "True"
            else:
                payload['last'] = "False"
            payload['activation'] = parameters['activations']['layer' + str(layer)]
            payloadbytes = dumps(payload)
            print("Payload to be sent to NeuronLambda: \n" + dumps(payload, indent=4, sort_keys=True))

            """
            # Invoke NeuronLambdas for next layer
            try:
                response = lambda_client.invoke(
                    FunctionName=parameters['ARNs']['NeuronLambda'],
                    InvocationType='Event',
                    Payload=payloadbytes
                )
            except botocore.exceptions.ClientError as e:
                sns_message = "Errors occurred invoking Neuron Lambda from TrainerLambda."
                sns_message += "\nError:\n" + e
                sns_message += "\nCurrent Payload:\n" +  dumps(payload, indent=4, sort_keys=True)
                publish_sns(sns_message)
                print(e)
                raise
            print(response)
            """

            return

    else:
        sns_message = "Errors processing `propogate()` function."
        publish_sns(sns_message)
        raise


def trainer_handler(event, context):
    """
    Processes the `event` vaiables from the various Lambda functions that call it, 
    i.e. `TrainerLambda` and `NeuronLambda`. Determines the "current" state and
    then directs the next steps.
    """    
    # Get the current state from the invoking lambda
    state = event.get('state')
    global parameters
    parameters = from_cache(endpoint=endpoint, key=event.get('parameter_key'))
    
    # Execute appropriate action based on the the current state
    if state == 'forward':
        # Get important state variables
        epoch = event.get('epoch')
        layer = event.get('layer')

        # First pre-process the Activations from the "previous" layer
        # Use the folling Redis command to ensure a pure string is return for the key
        r = redis(host=endpoint, port=6379, db=0, charset="utf-8", decode_responses=True)
        key_list = []
        # Compile a list of activations
        for key in r.scan_iter(match='layer'+str(layer-1)+'_a_*'):
            key_list.append(key)
        # Create a dictionary of activation results
        A_dict = {}
        for i in key_list:
            A_dict[i] = from_cache(endpoint=endpoint, key=i)
        # Number of Neuron Activations
        num_activations = len(key_list)
        # Create a numpy array of the results, depending on the number
        # of hidden units
        A = np.array([arr.tolist() for arr in A_dict.values()])
        dims = (key_list[0].split('|')[1].split('#')[1:])
        A = A.reshape(int(dims[0]), int(dims[1]))
        A_name = 'A' + str(layer-1)
        parameters['data_keys'][A_name] = to_cache(endpoint=endpoint, obj=A, name=A_name)

        # Update ElastiCache with this funciton's data
        parameter_key = to_cache(endpoint=endpoint, obj=parameters, name='parameters')
        
        # Determine the location within forwardprop
        if layer > parameters['layers']:
            # Location is at the end of forwardprop (layer 3), therefore calculate Cost
            # Get the training examples data
            Y = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_y'])
            #m = Y.shape[1]
            m = from_cache(endpoint=endpoint, key=parameters['data_keys']['m'])
            
            # Calculate the Cost (OLD)
            #cost = -1 / m * np.sum(np.multiply(Y, np.log(A)) + np.multiply((1 - Y), np.log(1 - A)))
            #cost = np.squeeze(cost)
            #assert(cost.shape == ())
            cost = (-1 / m) * np.sum(Y * (np.log(A)) + ((1 - Y) * np.log(1 - A)))

            # Update results with the Cost
            # Get the results object
            cost2results = from_cache(endpoint=endpoint, key=parameters['data_keys']['results'])
            # Append the cost to results object
            cost2results['epoch' + str(epoch)]['cost'] = cost
            # Update results key in ElastiCache
            parameters['data_keys']['results'] = to_cache(endpoint=endpoint, obj=cost2results, name='results')

            print("Cost after epoch {0}: {1}".format(epoch, cost))

            # Initialize backprop
            """
            Note: Not going to calculate dZ
            # Calculate the derivative of the Cost with respect to the last activation
            # Ensure that `Y` is the correct shape as the last activation
            Y = Y.reshape(A.shape)
            dZ = - (np.divide(Y, A) - np.divide(1 - Y, 1 - A))
            dZ_name = 'dZ' + str(layer-1)
            parameters['data_keys'][dZ_name] = to_cache(endpoint=endpoint, obj=dZ, name=dZ_name)

            # Update parameters from theis function in ElastiCache
            parameter_key = to_cache(endpoint=endpoint, obj=parameters, name='parameters')
            """

            # Start Backpropogation
            # This should start with layer (layers = 3-1)
            propogate(direction='backward', epoch=epoch, layer=layer-1, parameter_key=parameter_key)
            
        else:
            # Move to the next hidden layer
            #debug
            print("Propogating forward onto Layer " + str(layer))
            propogate(direction='forward', epoch=epoch, layer=layer, parameter_key=parameter_key)
        
    elif state == 'backward':
        # Get important state variables
        epoch = event.get('epoch')
        layer = event.get('layer')
        
        # Determine the location within backprop
        if epoch == parameters['epochs']-1 and layer == 0:
            # Location is at the end of the final epoch
            # Retieve the "params"
            learning_rate = parameters['learning_rate']
            w = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['weights']
            )
            b = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['bias']
            )

            # Retrieve the gradients
            grads = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['grads']
            )
            dw = from_cache(
                endpoint=endpoint,
                key=grads['layer'+ str(layer + 1)]['dw']
            )
            db = from_cache(
                endpoint=endpoint,
                key=grads['layer'+ str(layer + 1)]['db']
            )

            # Run Gradient Descent
            w = w - learning_rate * dw
            b = b - learning_rate * db

            # Update ElastiCache with the Weights and Bias so be used as the inputs for
            # the next epoch
            parameters['data_keys']['weights'] = to_cache(
                endpoint=endpoint,
                obj=w,
                name='weights'
            )
            parameters['data_keys']['bias'] = to_cache(
                endpoint=endpoint,
                obj=b,
                name='bias'
            )
            
            # Update paramters for the next epoch
            parameter_key = to_cache(
                endpoint=endpoint,
                obj=parameters,
                name='parameters'
            )
                        
            # Finalize the the process and clean up
            end(parameter_key=parameter_key)
            
        elif epoch < parameters['epochs']-1 and layer == 0:
            # Location is at the end of the current epoch and backprop is finished
            # Retieve the "params"
            learning_rate = parameters['learning_rate']
            w = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['weights']
            )
            b = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['bias']
            )

            # Retrieve the gradients
            grads = from_cache(
                endpoint=endpoint,
                key=parameters['data_keys']['grads']
            )
            dw = from_cache(
                endpoint=endpoint,
                key=grads['layer'+ str(layer + 1)]['dw']
            )
            db = from_cache(
                endpoint=endpoint,
                key=grads['layer'+ str(layer + 1)]['db']
            )

            # Run Gradient Descent
            w = w - learning_rate * dw
            b = b - learning_rate * db

            # Update ElastiCache with the Weights and Bias so be used as the inputs for
            # the next epoch
            parameters['data_keys']['weights'] = to_cache(
                endpoint=endpoint,
                obj=w,
                name='weights'
            )
            parameters['data_keys']['bias'] = to_cache(
                endpoint=endpoint,
                obj=b,
                name='bias'
            )
            
            # Update paramters for the next epoch
            parameter_key = to_cache(
                endpoint=endpoint,
                obj=parameters,
                name='parameters'
            )
                        
            # Start the next epoch
            start_epoch(epoch=epoch+1, layer=0, parameter_key=parameter_key)
            
        else:
            # Move to the next hidden layer
            propogate(direction='backward', epoch=epoch, layer=layer, parameter_key=parameter_key)
            
    elif state == 'start':
        # Start of a new run of the process        
        # Create initial parameters
        epoch = 0
        layer = 0
        start_epoch(epoch=epoch, layer=layer, parameter_key=event.get('parameter_key'))
       
    else:
        sns_message = "General error processing TrainerLambda handler!"
        publish_sns(sns_message)
        raise

In [5]:
event = {"state": "start", "parameter_key": "parameters|json"}
trainer_handler(event, context)

Starting Forward Propogation for epoch 0, layer 1
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


In [6]:
# Import Libraries needed by the Lambda Function
import numpy as np
import h5py
import scipy
import os
from os import environ
import json
from json import dumps, loads
from boto3 import client, resource, Session
import botocore
import uuid
import io
import redis
from redis import StrictRedis as redis

# Global Variables
#rgn = environ['Region']
s3_client = client('s3', region_name=rgn) # S3 access
s3_resource = resource('s3')
sns_client = client('sns', region_name=rgn) # SNS
redis_client = client('elasticache', region_name=rgn)
lambda_client = client('lambda', region_name=rgn) # Lambda invocations
# Retrieve the Elasticache Cluster endpoint
cc = redis_client.describe_cache_clusters(ShowCacheNodeInfo=True)
endpoint = cc['CacheClusters'][0]['CacheNodes'][0]['Endpoint']['Address']
cache = redis(host=endpoint, port=6379, db=0)

# Helper Functions
def publish_sns(sns_message):
    """
    Publish message to the master SNS topic.

    Arguments:
    sns_message -- the Body of the SNS Message
    """

    print("Publishing message to SNS topic...")
    sns_client.publish(TargetArn=environ['SNSArn'], Message=sns_message)
    return

def to_cache(endpoint, obj, name):
    """
    Serializes multiple data type to ElastiCache and returns
    the Key.
    
    Arguments:
    endpoint -- The ElastiCache endpoint
    obj -- the object to srialize. Can be of type:
            - Numpy Array
            - Python Dictionary
            - String
            - Integer
    name -- Name of the Key
    
    Returns:
    key -- For each type the key is made up of {name}|{type} and for
           the case of Numpy Arrays, the Length and Widtch of the 
           array are added to the Key.
    """
    
    # Test if the object to Serialize is a Numpy Array
    if 'numpy' in str(type(obj)):
        array_dtype = str(obj.dtype)
        if len(obj.shape) == 0:
            length = 0
            width = 0
        else:
            length, width = obj.shape
        # Convert the array to string
        val = obj.ravel().tostring()
        # Create a key from the name and necessary parameters from the array
        # i.e. {name}|{type}#{length}#{width}
        key = '{0}|{1}#{2}#{3}'.format(name, array_dtype, length, width)
        # Store the binary string to Redis
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a string
    elif type(obj) is str:
        key = '{0}|{1}'.format(name, 'string')
        val = obj
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is an integer
    elif type(obj) is int:
        key = '{0}|{1}'.format(name, 'int')
        # Convert to a string
        val = str(obj)
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    # Test if the object to serialize is a dictionary
    elif type(obj) is dict:
        # Convert the dictionary to a String
        val = json.dumps(obj)
        key = '{0}|{1}'.format(name, 'json')
        cache = redis(host=endpoint, port=6379, db=0)
        cache.set(key, val)
        return key
    else:
        sns_message = "`to_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported serialization type")
        raise

def from_cache(endpoint, key):
    """
    De-serializes binary object from ElastiCache by reading
    the type of object from the name and converting it to
    the appropriate data type
    
    Arguments:
    endpoint -- ElastiCacheendpoint
    key -- Name of the Key to retrieve the object
    
    Returns:
    obj -- The object converted to specifed data type
    """
    
    # Check if the Key is for a Numpy array containing
    # `float64` data types
    if 'float64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        if int(length) == 0:
            obj = np.float64(np.fromstring(val))
        else:
            obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a Numpy array containing
    # `int64` data types
    elif 'int64' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        val = cache.get(key)
        # De-serialize the value
        array_dtype, length, width = key.split('|')[1].split('#')
        obj = np.fromstring(val, dtype=array_dtype).reshape(int(length), int(width))
        return obj
    # Check if the Key is for a json type
    elif 'json' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return json.loads(obj)
    # Chec if the Key is an integer
    elif 'int' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return int(obj)
    # Check if the Key is a string
    elif 'string' in key:
        cache = redis(host=endpoint, port=6379, db=0)
        obj = cache.get(key)
        return obj
    else:
        sns_message = "`from_cache` Error:\n" + str(type(obj)) + "is not a supported serialization type"
        publish_sns(sns_message)
        print("The Object is not a supported de-serialization type")
        raise

def sigmoid(z):
    """
    Computes the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size

    Return:
    s -- sigmoid(z)
    """

    s = 1 / (1 + np.exp(-z))

    return s

def relu(z):
    """
    Implement the ReLU function.

    Arguments:
    z -- Output of the linear layer, of any shape

    Returns:
    a -- Post-activation parameter, of the same shape as z
    """

    a = np.maximum(0, z)

    assert(A.shape == z.shape)

    return a

def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def neuron_handler(event, context):
    """
    This Lambda Funciton simulates a single Perceptron for both 
    forward and backward propogation.
    """
    
    # Get the Neural Network paramaters from Elasticache
    parameters = from_cache(endpoint, key=event.get('parameter_key'))
       
    # Get the current state
    state = event.get('state')
    epoch = event.get('epoch')
    layer = event.get('layer')
    ID = event.get('id') # To be used when multiple activations
    # Determine is this is the last Neuron in the layer
    last = event.get('last')

    # Get data to process
    #w = from_cache(endpoint=endpoint, key=parameters['data_keys']['weights'])
    #b = from_cache(endpoint=endpoint, key=parameters['data_keys']['bias'])
    #X = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_x'])
    #Y = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_y'])
    #m = from_cache(endpoint=endpoint, key=parameters['data_keys']['m'])

    if state == 'forward':
        # Forward propogation from X to Cost
        activation = event.get('activation')
        X = from_cache(endpoint=endpoint, key=parameters['data_keys']['train_set_x'])
        w = from_cache(endpoint=endpoint, key=parameters['data_keys']['weights'])
        b = from_cache(endpoint=endpoint, key=parameters['data_keys']['bias'])
        if activation == 'sigmoid':
            a = sigmoid(np.dot(w.T, X) + b) # Single Neuron activation
        else:
            # No other functions supported on single layer at this time
            pass
        
        # Upload the results to ElastiCache for `TrainerLambda` to process
        to_cache(endpoint=endpoint, obj=a, name='layer'+str(layer)+'_a_'+str(ID))
        
        if last == "True":
            # Update parameters with this Neuron's data
            parameters['epoch'] = epoch
            parameters['layer'] = layer + 1
            # Build the state payload
            payload = {}
            payload['parameter_key'] = to_cache(endpoint=endpoint, obj=parameters, name='parameters')
            payload['state'] = 'forward'
            payload['epoch'] = epoch
            payload['layer'] = layer + 1
            payloadbytes = dumps(payload)
            print("Payload to be sent to TrainerLambda: \n" + dumps(payload, indent=4, sort_keys=True))

            """# Invoke TrainerLambda to process activations
            try:
                response = lambda_client.invoke(
                    FunctionName=parameters['ARNs']['TrainerLambda'],
                    InvocationType='Event',
                    Payload=payloadbytes
                )
            except botocore.exceptions.ClientError as e:
                sns_message = "Errors occurred invoking Trainer Lambd from NeuronLambdaa."
                sns_message += "\nError:\n" + e
                sns_message += "\nCurrent Payload:\n" +  dumps(payload, indent=4, sort_keys=True)
                publish_sns(sns_message)
                print(e)
                raise
            print(response)
            """

        return

    elif state == 'backward':
        # Backprop from Cost to X (A0)
        activation = event.get('activation')
        """
        Note: Not going to use dZ
        dZ_name = 'dZ' + str(layer)
        dZ = from_cache(
            endpoint=endpoint,
            key=parameters['data_keys'][dZ_name]
        )
        """
        # Get the activaion from the pcurrent layer
        A = from_cache(
            endpoint=endpoint,
            key=parameters['data_keys']['A' + str(layer)]
        )
        m = from_cache(
           endpoint=endpoint,
           key=parameters['data_keys']['m']
        )
        X = from_cache(
            endpoint=endpoint,
            key=parameters['data_keys']['train_set_x']
        )
        Y = from_cache(
            endpoint=endpoint,
            key=parameters['data_keys']['train_set_y']
        )
        # Backward propogation to determine gradients of current layer
        dw = (1 / m) * np.dot(X, (A - Y).T)
        db = (1 / m) * np.sum(A - Y)

        # Debug
        w = from_cache(endpoint=endpoint, key=parameters['data_keys']['weights'])
        assert(dw.shape == w.shape)
       
        # Capture gradients
        # Load the grads object
        grads = from_cache(endpoint, key=parameters['data_keys']['grads'])
        # Update the grads object with the calculated derivatives
        grads['layer' + str(layer)]['dw'] = to_cache(
            endpoint=endpoint,
            obj=dw,
            name='dw'
        )
        grads['layer' + str(layer)]['db'] = to_cache(
            endpoint=endpoint,
            obj=db,
            name='db'
        )
        # Update the pramaters (local)
        parameters['data_keys']['grads'] = to_cache(
            endpoint=endpoint,
            obj=grads,
            name='grads'
        )

        if last == "True":
            # Update parameters with this Neuron's data
            parameters['epoch'] = epoch
            parameters['layer'] = layer - 1
            # Build the state payload
            payload = {}
            payload['parameter_key'] = to_cache(endpoint=endpoint, obj=parameters, name='parameters')
            payload['state'] = 'backward'
            payload['epoch'] = epoch
            payload['layer'] = layer - 1
            payloadbytes = dumps(payload)
            print("Payload to be sent to TrainerLambda: \n" + dumps(payload, indent=4, sort_keys=True))

            """# Invoke NeuronLambdas for next layer
            try:
                response = lambda_client.invoke(
                    FunctionName=parameters['ARNs']['TrainerLambda'], #ENSURE ARN POPULATED BY CFN
                    InvocationType='Event',
                    Payload=payloadbytes
                )
            except botocore.exceptions.ClientError as e:
                sns_message = "Errors occurred invoking Trainer Lambda from NauronLambda."
                sns_message += "\nError:\n" + e
                sns_message += "\nCurrent Payload:\n" +  dumps(payload, indent=4, sort_keys=True)
                publish_sns(sns_message)
                print(e)
                raise
            print(response)
            """

        return

    else:
        sns_message = "General error processing NeuronLambda handler."
        publish_sns(sns_message)
        raise

In [7]:
event={
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


In [8]:
event = {
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer_handler(event, context)

Cost after epoch 0: 0.6931471805599453
Starting Backward Propogation for epoch 0, layer 1
Payload to be sent to NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [9]:
event = {
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [10]:
event = {
    "epoch": 0,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}
trainer_handler(event, context)

Starting Forward Propogation for epoch 1, layer 1
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 1,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
## Epoch 1

In [11]:
event = {
    "activation": "sigmoid",
    "epoch": 1,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 1,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


In [12]:
event = {
    "epoch": 1,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer_handler(event, context)

Cost after epoch 1: 0.7410294145065183
Starting Backward Propogation for epoch 1, layer 1
Payload to be sent to NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 1,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [13]:
event = {
    "activation": "sigmoid",
    "epoch": 1,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 1,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [14]:
event = {
    "epoch": 1,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}
trainer_handler(event, context)

Starting Forward Propogation for epoch 2, layer 1
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 2,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---

## Epoch 2

In [15]:
event = {
    "activation": "sigmoid",
    "epoch": 2,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 2,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


In [16]:
event = {
    "epoch": 2,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer_handler(event, context)

Cost after epoch 2: 0.753153581886211
Starting Backward Propogation for epoch 2, layer 1
Payload to be sent to NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 2,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [17]:
event = {
    "activation": "sigmoid",
    "epoch": 2,
    "id": 1,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "backward"
}
neuron_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 2,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}


In [18]:
event = {
    "epoch": 2,
    "layer": 0,
    "parameter_key": "parameters|json",
    "state": "backward"
}
trainer_handler(event, context)

In [19]:
# Get the results key
final_results = from_cache(
    endpoint=endpoint,
    key=parameters['data_keys']['results']
)
final_results

{'epoch0': {'cost': 0.6931471805599453},
 'epoch1': {'cost': 0.7410294145065183},
 'epoch2': {'cost': 0.753153581886211}}